In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

import pandas as pd
import csv
import numpy as np

### Load Artists file

Let's load the artist TSV file.  This will load the names of the artists.  We are not currently using this file right now, but may use it in the future.

There are many errors in the file; we'll just skip them.


In [ ]:
#artists = pd.read_csv("/data/audioscrobble/artist_data.txt.gz", sep='\t', error_bad_lines=False, na_values=['na'], warn_bad_lines=False, dtype={'ArtistID':np.int32, 'ArtistName':str})
#artists.describe()

### Load User Artists Data file

Let's load the user artist Data file -- this is our main data. And it's pretty clean.

Notice that what we have is the playcount.  That's not the same as a rating.  We'll need to decide how we deal with this.

In [ ]:
dataset = pd.read_csv("/data/audioscrobble/user_artist_data.csv.gz")
#dataset = pd.merge(dataset, artists, how = 'left', left_on='Artist', right_on='ArtistID')
dataset

### Let's load the artist alias file

This is a file that has artist aliases.  Misspellings, alternate versions, etc.  This will help us clean up our data.

We'll load this into a a dictionary of str:str

In [ ]:
alias = {}
reader = csv.reader(open('/data/audioscrobble/artist_alias.txt', 'r'), delimiter='\t')
alias = dict(reader)
alias

In [ ]:
# Clean up the dataset artist by using the dictionary.
dataset['Artist'] = dataset['Artist'].apply(lambda x: int(alias[str(x)]) if str(x) in alias else x)

### Let's look at a summary of the data

* ** TODO: Do a describe() on the data **

In [ ]:
### TODO: DO a describe() on the dataset()
dataset.describe()

### Decide what to do with the playcount

How are we going to deal with the playcount?  Presumably, if the person likes the music, they will play it more. If they only play it once or twice, maybe they didn't like it?  Hard to say, but we have to do something with it.

Here's a proposal.  We'll treat the playcount as a "rating" from 1 to 5.  Playcounts of higher than 5 will just be 5. Presumably if someone who plays a song more than 5 times likes it.

We then have to set the "scale" of the rating.  Here from 1 to 5.

* ** TODO: What is your idea? Do you have a better way of treating the playcount?  **

In [ ]:
dataset['Rating'] = dataset['PlayCount'].apply(lambda x: 5 if x > 5 else x)  # Playcounts > 5 are treated as 5.

# A reader is still needed but only the rating_scale param is required.
reader = Reader(rating_scale=(1, 5))

### TODO: Think of a way todo the Playcount.


### Convert pandas dataframe to Dataset format

We need to use the internal scikit-surprise format for the data.  Luckily, it's easy to convert from a pandas dataframe.

In [ ]:
# Load the audioscrobbler dataset
data = Dataset.load_from_df(dataset[['User', 'Artist', 'Rating']], reader)


### Use the SVD Algorithm to train the model.

Note: This may take a LOOONNG time to run.
    
We will run a cross validation while we train and get the results.

In [ ]:
# Run 5-fold cross-validation and print results.
cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

### Evaluate the Results

What is the RMSE?  What does that tell us about the results?
